In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from einops import rearrange

In [2]:
from ezflow.models import MODEL_REGISTRY
from ezflow.config import configurable
from ezflow.modules import BaseModule

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

___
1. https://huggingface.co/blog/perceiver
2. https://huggingface.co/docs/transformers/v4.21.3/en/model_doc/perceiver
___
3. https://huggingface.co/docs/transformers/v4.21.3/en/model_doc/perceiver#transformers.PerceiverForOpticalFlow
4. https://github.com/huggingface/transformers/blob/v4.21.3/src/transformers/models/perceiver/modeling_perceiver.py#L1612
5. https://github.com/NielsRogge/Transformers-Tutorials/blob/master/Perceiver/Perceiver_for_Optical_Flow.ipynb
___

In [4]:
from transformers import PerceiverModel, PerceiverConfig
from transformers import PerceiverForOpticalFlow
from transformers.models.perceiver.modeling_perceiver import PerceiverImagePreprocessor, PerceiverOpticalFlowDecoder

In [5]:
def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

___

### Pretrained

In [13]:
patches = torch.randn(1,2,27,368, 496)

In [14]:
pretrained_model = PerceiverForOpticalFlow.from_pretrained("deepmind/optical-flow-perceiver")

In [8]:
count_params(pretrained_model)

41057134

In [9]:
# pretrained_model = pretrained_model.to(device)

In [10]:
# patches = patches.to(device)

In [11]:
patches.device

device(type='cpu')

In [15]:
outputs = pretrained_model(inputs=patches, return_dict=False)

flow = outputs[0]

In [16]:
flow.shape

torch.Size([1, 368, 496, 2])

In [17]:
del pretrained_model, flow, outputs

___

In [15]:
class Perceiver(BaseModule):
    """
    Implementation of PerceiverIO Optical Flow
    https://www.deepmind.com/open-source/perceiver-io
    https://huggingface.co/docs/transformers/v4.21.3/en/model_doc/perceiver#transformers.PerceiverForOpticalFlow
    https://github.com/huggingface/transformers/blob/v4.21.3/src/transformers/models/perceiver/modeling_perceiver.py#L1612
    

    Parameters
    ----------
    cfg : :class:`CfgNode`
        Configuration for the model
    """

    def __init__(self, cfg):
        super(Perceiver, self).__init__()
        self.cfg = cfg
        
        config = PerceiverConfig(**cfg)
        
        fourier_position_encoding_kwargs_preprocessor = dict(
            num_bands=64,
            max_resolution=config.train_size,
            sine_only=False,
            concat_pos=True,
        )
        fourier_position_encoding_kwargs_decoder = dict(
            concat_pos=True, max_resolution=config.train_size, num_bands=64, sine_only=False
        )
        
        image_preprocessor = PerceiverImagePreprocessor(
            config,
            prep_type="patches",
            spatial_downsample=1,
            conv_after_patching=True,
            conv_after_patching_in_channels=54,
            temporal_downsample=2,
            position_encoding_type="fourier",
            # position_encoding_kwargs
            fourier_position_encoding_kwargs=fourier_position_encoding_kwargs_preprocessor,
        )
        
        self.perceiver = PerceiverModel(
            config,
            input_preprocessor=image_preprocessor,
            decoder=PerceiverOpticalFlowDecoder(
                config,
                num_channels=image_preprocessor.num_channels,
                output_image_shape=config.train_size,
                rescale_factor=100.0,
                use_query_residual=False,
                output_num_channels=2,
                position_encoding_type="fourier",
                fourier_position_encoding_kwargs=fourier_position_encoding_kwargs_decoder,
            ),
        )
        
    
    def _extract_patches(self, img, kernel_size=(3,3), stride=(1,1)):
        img = F.pad(img, (1,1,1,1), mode="constant")
        patches = img.unfold(2, kernel_size[0], stride[0]).unfold(3, kernel_size[1], stride[1])
        patches = rearrange(patches, 'n c h w k1 k2 -> n (c k1 k2) h w')
        #patches = patches.contiguous()
        return patches
        
    def forward(self, img1, img2):
        patches1 = self._extract_patches(img1)
        patches2 = self._extract_patches(img2)
        
        patches1 = torch.unsqueeze(patches1, dim=1)
        patches2 = torch.unsqueeze(patches2, dim=1)
        
        patches = torch.concat([patches1, patches2], dim=1)
        
        perceiver_outputs = self.perceiver(
            inputs=patches
        )
        
        flow = rearrange(perceiver_outputs.logits, 'n h w c -> n c h w')
        
        output = {"flow_preds": flow}
        
        if self.training:
            return output
        
        output["flow_upsampled"] = flow
        
        return output
        

In [16]:
config_dict = {
      "_name_or_path": "deepmind/optical-flow-perceiver",
      "architectures": [
        "PerceiverForOpticalFlow"
      ],
      "attention_probs_dropout_prob": 0.1,
      "audio_samples_per_frame": 1920,
      "cross_attention_shape_for_attention": "kv",
      "cross_attention_widening_factor": 1,
      "d_latents": 512,
      "d_model": 322,
      "hidden_act": "gelu",
      "hidden_dropout_prob": 0.1,
      "image_size": 56,
      "initializer_range": 0.02,
      "layer_norm_eps": 1e-12,
      "max_position_embeddings": 2048,
      "model_type": "perceiver",
      "num_blocks": 1,
      "num_cross_attention_heads": 1,
      "num_frames": 16,
      "num_latents": 2048,
      "num_self_attends_per_block": 24,
      "num_self_attention_heads": 16,
      "output_shape": [
        1,
        16,
        224,
        224
      ],
      "qk_channels": None,
      "samples_per_patch": 16,
      "self_attention_widening_factor": 1,
      "seq_len": 2048,
      "torch_dtype": "float32",
      "train_size": [
        368,
        496
      ],
      "transformers_version": "4.21.3",
      "use_query_residual": True,
      "v_channels": None,
      "vocab_size": 262
}

In [17]:
model = Perceiver(config_dict)

In [18]:
count_params(model)

41057134

In [8]:
img1 = torch.randn(1, 3, 368, 496)
img2 = torch.randn(1, 3, 368, 496)

In [9]:
flow_outputs = model(img1, img2)

C:\Users\prajn\miniconda3\envs\ezflow\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:76] data. DefaultCPUAllocator: not enough memory: you tried to allocate 1495269376 bytes.

In [ ]:
flow = flow_outputs["flow_preds"]

___

### Build using Ezflow

In [18]:
from ezflow.models import build_model
from nnflow import Perceiver

In [19]:
_model = build_model('Perceiver', cfg_path='../configs/perceiver/models/perceiver.yaml', custom_cfg=True)

In [20]:
_model.cfg

{'_name_or_path': 'deepmind/optical-flow-perceiver',
 'architectures': ['PerceiverForOpticalFlow'],
 'attention_probs_dropout_prob': 0.1,
 'audio_samples_per_frame': 1920,
 'cross_attention_shape_for_attention': 'kv',
 'cross_attention_widening_factor': 1,
 'd_latents': 512,
 'd_model': 322,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'image_size': 56,
 'initializer_range': 0.02,
 'layer_norm_eps': 1e-12,
 'max_position_embeddings': 2048,
 'model_type': 'perceiver',
 'num_blocks': 1,
 'num_cross_attention_heads': 1,
 'num_frames': 16,
 'num_latents': 2048,
 'num_self_attends_per_block': 24,
 'num_self_attention_heads': 16,
 'output_shape': [1, 16, 224, 224],
 'qk_channels': None,
 'samples_per_patch': 16,
 'self_attention_widening_factor': 1,
 'seq_len': 2048,
 'torch_dtype': 'float32',
 'train_size': [368, 496],
 'transformers_version': '4.21.3',
 'use_query_residual': True,
 'v_channels': None,
 'vocab_size': 262}

In [7]:
img1 = torch.randn(1, 3, 368, 496)
img2 = torch.randn(1, 3, 368, 496)

In [22]:
# img1 = img1.to(device)
# img2 = img2.to(device)
# _model = _model.to(device)

In [23]:
flow_outputs = _model(img1, img2)

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:76] data. DefaultCPUAllocator: not enough memory: you tried to allocate 1495269376 bytes.

In [ ]:
flow = flow_outputs["flow_preds"]
flow.shape

In [16]:
import math

In [17]:
def extract_image_patches(x, kernel=3, stride=1, dilation=1):
        # Do TF 'SAME' Padding
        b,c,h,w = x.shape
        h2 = math.ceil(h / stride)
        w2 = math.ceil(w / stride)
        pad_row = (h2 - 1) * stride + (kernel - 1) * dilation + 1 - h
        pad_col = (w2 - 1) * stride + (kernel - 1) * dilation + 1 - w
        x = F.pad(x, (pad_row//2, pad_row - pad_row//2, pad_col//2, pad_col - pad_col//2))

        # Extract patches
        patches = x.unfold(2, kernel, stride).unfold(3, kernel, stride)
        patches = patches.permute(0,4,5,1,2,3).contiguous()

        return patches.view(b,-1,patches.shape[-2], patches.shape[-1])

In [12]:
imgs = torch.stack([img1, img2], dim=1)
imgs.shape

torch.Size([1, 2, 3, 368, 496])

In [14]:
b, _, c, h, w = imgs.shape
imgs = imgs.view(b*_, c, h, w)
imgs.shape

torch.Size([2, 3, 368, 496])

In [18]:
patches = extract_image_patches(imgs)
patches.shape

torch.Size([2, 27, 368, 496])

In [19]:
patches = patches.view(b, -1, c, h, w)
patches.shape

torch.Size([1, 18, 3, 368, 496])